In [0]:
# ==============================================================================
# PASSO 1: CARREGAR MODELOS TREINADOS E DADOS DAS PRÓXIMAS PARTIDAS
# ==============================================================================
from pyspark.ml import PipelineModel
from pyspark.sql import functions as F
from pyspark.ml.feature import IndexToString

# --- AJUSTE APLICADO AQUI: DEFININDO O CONTEXTO CORRETO ---
# Garante que este notebook saiba em qual catálogo procurar as tabelas. 💡
spark.sql("USE CATALOG previsao_brasileirao")
# --- FIM DO AJUSTE ---


# --- CARREGAR OS MODELOS TREINADOS ---
# ... (seu código para carregar modelos) ...


# --- CARREGAR DADOS DE PARTIDAS FUTURAS ---
# Agora o Spark sabe que 'bronze.partidas_raw' está em 'previsao_brasileirao'
df_futuro_raw = spark.table("bronze.partidas_raw") \
    .filter("placar_oficial_mandante IS NULL AND valida = true") \
    .orderBy("partida_data")

# --- CARREGAR DADOS HISTÓRICOS NECESSÁRIOS PARA FEATURES ---
df_partidas_historico = spark.table("gold.feature_store")
df_clubes = spark.table("silver.clubes")

print(f"Encontradas {df_futuro_raw.count()} partidas futuras para prever.")

In [0]:
# ==============================================================================
# PASSO 2: APLICAR A ENGENHARIA DE FEATURES NAS PARTIDAS FUTURAS
# ==============================================================================
# NOTA: Este é um processo complexo. O código abaixo é uma simplificação que usa
# as features mais recentes do nosso histórico como base para as partidas futuras.
# Um pipeline de produção teria a mesma lógica do Notebook 03_Gold.

# Pegar o Elo mais recente de cada time do nosso histórico
latest_elos = df_partidas_historico \
    .groupBy("mandante_id") \
    .agg(F.max("rodada").alias("rodada")) \
    .join(df_partidas_historico, ["mandante_id", "rodada"]) \
    .select("mandante_id", F.col("elo_mandante_pre_jogo").alias("elo_atual"))

# Pegar as médias móveis mais recentes de cada time
latest_mms_m = df_partidas_historico.groupBy("mandante_id").agg(F.max("rodada").alias("rodada")).join(df_partidas_historico, ["mandante_id", "rodada"]).select("mandante_id", "mm_gols_m", "mm_fin_m", "mm_des_m")
latest_mms_v = df_partidas_historico.groupBy("visitante_id").agg(F.max("rodada").alias("rodada")).join(df_partidas_historico, ["visitante_id", "rodada"]).select("visitante_id", "mm_gols_v", "mm_fin_v", "mm_des_v")

# Preparar as partidas futuras e juntar as features
df_futuro_features = df_futuro_raw.select(
    F.col("partida_id"),
    F.col("rodada"),
    F.col("clube_casa_id").alias("mandante_id"),
    F.col("clube_visitante_id").alias("visitante_id"),
    F.to_timestamp("partida_data").alias("data_partida")
)

# Juntar ELO
df_futuro_features = df_futuro_features.join(latest_elos, df_futuro_features.mandante_id == latest_elos.mandante_id, "left").withColumnRenamed("elo_atual", "elo_mandante_pre_jogo").drop("mandante_id")
df_futuro_features = df_futuro_features.join(latest_elos, df_futuro_features.visitante_id == latest_elos.mandante_id, "left").withColumnRenamed("elo_atual", "elo_visitante_pre_jogo").drop("mandante_id")

# Juntar MMs Mandante
df_futuro_features = df_futuro_features.join(latest_mms_m, df_futuro_features.mandante_id == latest_mms_m.mandante_id, "left").drop("mandante_id", "rodada")
# Juntar MMs Visitante
df_futuro_features = df_futuro_features.join(latest_mms_v, df_futuro_features.visitante_id == latest_mms_v.visitante_id, "left").drop("visitante_id", "rodada")

# Calcular diferenciais
df_futuro_features = df_futuro_features.withColumn("elo_diff", F.col("elo_mandante_pre_jogo") - F.col("elo_visitante_pre_jogo")) \
                                       .withColumn("diff_mm_gols", F.col("mm_gols_m") - F.col("mm_gols_v")) \
                                       .withColumn("diff_mm_fin", F.col("mm_fin_m") - F.col("mm_fin_v")) \
                                       .withColumn("diff_mm_des", F.col("mm_des_m") - F.col("mm_des_v")) \
                                       .na.fill(0)

print("Engenharia de features para dados futuros concluída.")

In [0]:
# ==============================================================================
# PASSO 3: FAZER AS PREVISÕES E FORMATAR A SAÍDA
# ==============================================================================
# Nível 1: Gerar previsões dos especialistas
pred_geral_futuro = modelo_geral.transform(df_futuro_features)
pred_lr_futuro = modelo_lr.transform(df_futuro_features)
pred_rf_futuro = modelo_rf.transform(df_futuro_features)

# Preparar as meta-features para o Nível 2
meta_features_geral = pred_geral_futuro.select("partida_id", F.col("probability").alias("prob_geral"))
meta_features_lr = pred_lr_futuro.select("partida_id", F.col("probability").alias("prob_lr"))
meta_features_rf = pred_rf_futuro.select("partida_id", F.col("probability").alias("prob_rf"))

df_inferencia_meta = df_futuro_features.select("partida_id") \
    .join(meta_features_geral, "partida_id") \
    .join(meta_features_lr, "partida_id") \
    .join(meta_features_rf, "partida_id")

df_inferencia_meta = assembler_meta.transform(df_inferencia_meta) # Reusa o 'assembler_meta' do notebook de treino

# Nível 2: Fazer a previsão final com o Meta-Modelo
previsoes_finais = modelo_final.transform(df_inferencia_meta)

# Converter o índice numérico da previsão de volta para texto (V/E/D)
converter = IndexToString(inputCol="prediction", outputCol="previsao_texto", labels=label_indexer.labels)
previsoes_texto = converter.transform(previsoes_finais)

# Juntar com os nomes dos clubes para a visualização final
df_resultado_final = df_futuro_features \
    .join(previsoes_texto.select("partida_id", "previsao_texto", "probability"), "partida_id") \
    .join(df_clubes.alias("mandante"), F.col("mandante_id") == F.col("mandante.clube_id")) \
    .join(df_clubes.alias("visitante"), F.col("visitante_id") == F.col("visitante.clube_id")) \
    .withColumn("competicao", F.lit("Brasileirão Série A")) \
    .select(
        "competicao",
        F.date_format("data_partida", "dd/MM/yyyy HH:mm").alias("data_hora_partida"),
        F.col("mandante.nome").alias("time_mandante"),
        F.col("visitante.nome").alias("time_visitante"),
        F.col("previsao_texto").alias("previsao"),
        F.col("probability").alias("confianca_probabilidades")
    ).orderBy("data_hora_partida")

# Salvar o resultado na Camada Diamond para consulta
df_resultado_final.write.mode("overwrite").format("delta").saveAsTable("diamond.previsoes_proximas_partidas")

print("Previsões geradas com sucesso!")
# Criar uma view temporária para facilitar a consulta via SQL
df_resultado_final.createOrReplaceTempView("visualizacao_previsoes")